<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/03_Upload_clips_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso_data_management and its requirements</font> { vertical-output: true }

from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/requirements.txt)
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False

    # Install requirements
    # %pip install -q --no-warn-script-location --upgrade pip --user
    # %pip install -qr ../requirements.txt --user
    # %pip install -qr ../kso_utils/requirements.txt --user

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension install --user --py ipysheet
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Import Python packages</font> { vertical-output: true }

# Set the directory of the libraries
import sys

try:
    if "kso_utils" not in sys.modules:
        sys.path.append("..")
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t3_utils as t3
from kso_utils.project import ProjectProcessor


print("Packages loaded successfully")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify GPU availability</font> { vertical-output: true }
gpu_available = t_utils.gpu_select()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }
project_name = t_utils.choose_project()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Initiate project's database</font> { vertical-output: true }
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)

# Initiate pp
pp = ProjectProcessor(project)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Connect to Zooniverse</font> { vertical-output: true }
pp.get_zoo_info()

# Select the movie to upload to Zooniverse

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify movie of interest</font> { vertical-output: true }
pp.preview_media()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check if movie is already in Zooniverse</font> { vertical-output: true }

# Remember to query the newest zooniverse data to get the most up to date list of clips uploaded
pp.check_movies_uploaded(pp.movie_selected)

# Create some clip examples (Optional)
Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
# @title <font size="5">↓ ឵឵<i>Create the clip examples</font> { vertical-output: true }
pp.generate_zu_clips(pp.movie_selected, pp.movie_path, is_example=True)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Review the size of the clips</font> { vertical-output: true }
t3.check_clip_size(clips_list=pp.generated_clips.clip_path)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Display clips side-by-side (in progress)</font> { vertical-output: true }
t3.compare_clips(
    example_clips=pp.generated_clips.clip_path,
    modified_clips=pp.generated_clips.modif_clip_path,
)

# Create the clips to upload to Zooniverse

In [ ]:
pp.generate_zu_clips(pp.movie_selected, pp.movie_path, is_example=False)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check the size of the clips</font> { vertical-output: true }
t3.check_clip_size(clips_list=pp.generated_clips.clip_path)

# Upload clips to Zooniverse
Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
# @title <font size="5">↓ ឵឵<i>Upload clips to Zooniverse</font> { vertical-output: true }
pp.upload_zu_subjects(pp.generated_clips, "clip")

In [ ]:
# END